# Data to Strategy with Alpaca
Query and process Alpaca OHLCV data. Add features, identify principal components and develop initial trade algorithm.
    
https://github.com/emskiphoto/Alpaca-Strategy-Development<BR>
Copyright 2022 Matt Chmielewski

#### References:
https://github.com/stefan-jansen/machine-learning-for-trading

https://github.com/twopirllc/pandas-ta

https://github.com/emskiphoto/alpaca-trade-api-python

### Load Dependencies

In [1]:
!pip3 install matplotlib
!pip3 install alpaca-trade-api as alpaca_trade_api
!pip3 install ta
!pip3 install pandas_ta
!pip3 install datetime
!pip3 install pandas
!pip3 install dotenv
print("cell 1 done")

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 36, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/private/var/folders/v0/7gtj3bhx74z6w7kx78fhqlmm0000gn/T/pip-wheel-hcfhfg9h/distribute_58777639f4284a05baa5266de136a297/setuptools/__init__.py", line 2, in <module>
                from setuptools.extension import Extension, Library
              File "/private/var/folders/v0/7gtj3b

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
# from alpaca_trade_api.rest import TimeFrame - Won't work 6-11-21
from datetime import datetime, timedelta
import timeit
import calendar
from alpaca_trade_api.rest import REST
import pandas_ta as ta
from ta import add_all_ta_features
from ta.utils import dropna
from dotenv import load_dotenv

load_dotenv()

# from alpaca_trade_api.rest import TimeFrame
# Disable jedi autocompleter
%config Completer.use_jedi = False
print("cell 2 done")

cell 2 done


#### Display and Plot options

In [3]:
pd.options.display.max_rows = 300
    # create idx for mutlti-index slicing
idx = pd.IndexSlice
plt.rcParams['figure.figsize'] = [14,6]
print("cell 3 done")

cell 3 done


### Check connection to Alpaca

#### Verify Windows Environment Variables are available
API Credentials are stored on Windows PATH variable.  https://dev.to/biplov/handling-passwords-and-secret-keys-using-environment-variables-2ei0


In [4]:
APCA_API_SECRET_KEY = 'ALr7od44fpwRvDyPktqj7PNPAo0jjy3T1NOz82Iw'
APCA_API_KEY_ID = 'PKW9VTG8TDIZ2RJJ80MB'
APCA_API_BASE_URL = 'https://data.alpaca.markets/v2'
API_credential_vars = ['APCA_API_BASE_URL','APCA_API_KEY_ID', 'APCA_API_SECRET_KEY']
# for var in API_credential_vars:
#     print(os.getenv(var))
print("cell 4 done")

cell 4 done


### Check Alpaca account

In [5]:
APCA_API_SECRET_KEY = 'ALr7od44fpwRvDyPktqj7PNPAo0jjy3T1NOz82Iw'
APCA_API_KEY_ID = 'PKW9VTG8TDIZ2RJJ80MB'
APCA_API_BASE_URL = 'https://data.alpaca.markets/v2'

api = tradeapi.REST()
# Get our account information.
account = api.get_account()
# Check if our account is restricted from trading.
api

In [6]:

if account.trading_blocked:
    print('Account is currently restricted from trading.')
# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))
print("cell 5 done")

$200000 is available as buying power.
cell 5 done


### Download Price History

#### Get list of assets

In [7]:
def build_asset_df_alpaca(api, status='active'):
    """Returns a list of assets and trading characteristics as a dataframe"""
    assets = api.list_assets(status=status)
    return pd.DataFrame.from_records([asset._raw for asset in assets.__iter__()])
df_assets = build_asset_df_alpaca(api)
df_assets.head(2)
# print("cell 6 done")

,id,class,exchange,symbol,name,status,tradable,marginable,maintenance_margin_requirement,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment
0,8444480d-d13a-4fa9-adde-6225431d1753,us_equity,OTC,REVRQ,REVLON INC Common Stock,active,False,False,100,False,False,False,NaN,NaN,NaN
1,4e872f2e-f5ad-417b-9f12-832fb16c03cb,us_equity,OTC,BRCNF,Burcon NutraScience Corp.,active,False,False,100,False,False,False,NaN,NaN,NaN


In [8]:
df_assets.groupby(['exchange','marginable']).count().iloc[:,:2]
# print("cell 7 done")

,,id,class
exchange,marginable,,
AMEX,True,280,280
ARCA,True,1936,1936
BATS,True,575,575
FTXU,False,52,52
NASDAQ,True,4995,4995
NYSE,True,2871,2871
OTC,False,375,375


In [9]:
df_assets['class'].value_counts()
# print("cell 8 done")

us_equity    11032
crypto          52
Name: class, dtype: int64

## Build OHLCV Database
https://alpaca.markets/docs/api-documentation/api-v2/market-data/alpaca-data-api-v2/

### Set list of Assets

In [10]:
print(f'There are {df_assets["symbol"].nunique()} assets total')
# assets_query = df_assets['symbol'].sample(10).sort_values().to_list()
assets_query = df_assets['symbol'][:100].sort_values().to_list()
print(f'There are {len(assets_query)} assets in assets_query')
assets_query[:10]
# print("cell 9 done")

There are 11084 assets total
There are 100 assets in assets_query


['ACHHY',
 'ACMSY',
 'ACUT',
 'ADXS',
 'AFIIQ',
 'ALFIQ',
 'ALJJ',
 'AMPE',
 'ARGGY',
 'AXAS']

The rate limit is 200 requests every minute per API key.<BR>
All date time type inputs and outputs are serialized according to ISO8601<BR>


In [11]:
# custom function
def build_multi_symbol_df(assets,start_date = '2021-01-01', end_date = '2021-05-25',
                            frequency='15Min', TZ = 'America/New_York', limit=1000):
    """Returns single multi-index OHLCV time series dataframe for list of assets over input time
    range using Alpaca's API 'get_bars()' method.  'assets' must be input as list.
    Output multi-index is:  symbol, timestamp
    and output columns are: open, high, low, close, volume.
   
    frequency options:
    Day = "1Day"
    Hour = "1Hour"
    15 Minute = '15Min'
    Minute = "1Min"
    Sec = "1Sec"
    
    """
    # reference problematic query (missing bars - not used here):
    # TZ = 'America/New_York'
    # bars = api.get_bars('AAQC.U', '5Min',
                         # end=pd.Timestamp('2021-05-19', tz=TZ).isoformat(),
                           # limit=1000).df.droplevel(0,axis=1)
    # Sort assets for the benefit of future indexing
    assets = sorted(assets)
    # Get Name of columns for temp df
    n_assets = len(assets)
    print(f'{n_assets} total assets to query')
    start_date_plus_1 = datetime.strftime(datetime.strptime(start_date,"%Y-%m-%d") + timedelta(days=10),"%Y-%m-%d" )

    # Build empty DataFrame
    # Get names of columns based on short query of first asset
    columns = list(api.get_bars(assets[0], frequency,
        start=start_date, end=start_date_plus_1).df.droplevel(0,axis=1).columns)

    # Create empty dataframe as multiindex to hold bars responses
    temp = pd.DataFrame(columns=['symbol',
             'timestamp'] + columns).set_index(['symbol','timestamp']).asfreq(frequency)
    bars_ = []
    bars = []

    # Set dates
    end_date_range = pd.to_datetime(pd.Timestamp(end_date, tz=TZ).isoformat())
    start_date_dt = pd.to_datetime(pd.Timestamp(start_date, tz=TZ).isoformat())
    print(start_date_dt, "\n", end_date_range)

    # Iterate through assets:
    for idx, symbol in enumerate(assets):
        # update start_date_dt
        print(f'== {idx + 1} of {n_assets} querying "{symbol}" from {start_date_dt} to {end_date_range} ==')

        # limit date range to not exceed API's bars limit
        count = 0
        end_date_dt = end_date_range
        last_index_start = end_date_range
        while end_date_dt.to_pydatetime() >= start_date_dt.to_pydatetime():
            count+=1
            # print(f'end Date is: {end_date_dt}')
            print(f'Chunk {count} - {symbol} desired end date:  {end_date_dt}')

            # Chunked query starts with end date and works backwards in time to start date
            bars = api.get_bars(symbol, frequency,
                                  end=end_date_dt.isoformat(),
                                  limit=limit).df.droplevel(0,axis=1)

            print(f'Bars shape:  {bars.shape}')
            # Check if bars is empty, if so break out of loop
            try:
                bars.index[0]
            except:
                break
            # Break out of loop if start date repeats (indicating no history prior to start date available)
            if bars.index[0].to_pydatetime() == last_index_start:
                print(f'<<<<<  No history prior to {last_index_start} available for {symbol}  >>>>>\n\n')
                break
            last_index_start = bars.index[0].to_pydatetime()
            print(f'{symbol} chunk {count} index start:  {bars.index[0]}')
            # print(f'Start of barset index {bars.index[0]}')
            print(f'{symbol} chunk {count} index end:   {bars.index[-1]}')
            # If an empty 'bars' is returned, skip to next asset
            if len(bars.index) < 1:
                break
            
            # end of date range based on chunk size:
            # For testing:
            # start_date_dt = datetime.strptime(start_date_str,"%Y-%m-%d") + timedelta(days=10)

            # Reset chunk end date to be one period before the start of the last chunk that was queried
            end_date_dt = pd.Timestamp(bars.index[0]) - pd.Timedelta(frequency)
            print(f'Is {end_date_dt.to_pydatetime()} less than {start_date_dt.to_pydatetime()}'
                  f': {end_date_dt.to_pydatetime() < start_date_dt.to_pydatetime()}\n')
            # slice off records with dates outside of input start and end dates and set index frequency of df
            # bars = bars.loc[start_date:end_date]
            # bars = bars.loc[start_date:end_date]
            bars = bars.loc[start_date:end_date].asfreq(frequency).sort_index()
            # print(bars.index)
            # Create multi-index as list of tuples
            unique_timestamps = bars.index.unique()
            idx_tuples = [(symbol,timestamp) for timestamp in unique_timestamps]
            # idx_tuples = [(symbol,timestamp) for timestamp in bars.index]
            bars.set_index(pd.Index(idx_tuples), inplace=True)
            bars.rename_axis(['symbol','timestamp'], inplace=True)
            bars_.append(bars)
        
            # Drop duplicate indices, if needed
            # unique_idx = bars_.index.drop_duplicates(keep='first')
            # bars_ = bars.loc[unique_idx]
        
    # completed compiling bars_, now commit to dataframe
    df = pd.concat([temp] + bars_)

    # set frequency of datetime index
    df.index.levels[1].freq = frequency

    # If there is a volume column fill any NaN and convert to integer
    if 'volume' in df.columns:
        # df['volume'] = df['volume'].interpolate().astype('int')
        df['volume'] = df['volume'].astype('int', errors='ignore')
        # df.index = df.index.tz_convert('America/New_York')
        # Sort symbols and timestamps alphabetically and chronologically

        # df.sort_index(level=['symbol','timestamp'], inplace=True)

    # remove records where OHLC are all NaN
    df = df.dropna(subset=['open','high','low','close'], how='all')

    return df
    del bars_, df
print("cell 10 done")

cell 10 done


### Query assets

#### Asset query parameters
In the future, implement a 'class' to manage attributes of queries

In [12]:
frequency = '15Min'
start_date = '2020-01-01'
end_date = '2021-05-30'
TZ = 'America/New_York'
OHLC_cols = ['open','high','low','close']
print("cell 11 done")

cell 11 done


In [13]:
%timeit

# list of particularly messy symbols:
test_list = ['ACSI', 'AHT', 'TMAT', 'AROC', 'BBUS', 'CAT', 'AMAM']

# df = build_multi_symbol_df(assets_query[:10],start_date = '2020-01-01', end_date = '2021-05-25',
                # frequency='5Min', TZ = 'America/New_York', limit=1000)
df = build_multi_symbol_df(test_list, start_date = start_date, end_date = end_date, frequency=frequency, TZ = TZ, limit=1000)
print("cell 12 done")

7 total assets to query


ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

#### Save df to disk

In [ ]:
# df.to_csv(f'data/{df.index.levels[0].nunique()}_symbols_{start_date}_to_{end_date}_raw.csv')

#### Was full time range of data queried?

In [ ]:
start_date == datetime.strftime(df.index.levels[1].min(), "%Y-%m-%d")

In [ ]:
end_date == datetime.strftime(df.index.levels[1].max(), "%Y-%m-%d")

In [ ]:
start_date, df.index.levels[1].min(), end_date, df.index.levels[1].max()

#### Are there gaps in the dates or times of the time series?

In [ ]:
symbols_by_close_count = df.groupby(level=0).count().sort_values(by='close').index

In [ ]:
df.loc[idx[symbols_by_close_count[0],:]].head(6)

In [ ]:
# Are all index values spaced evenly?
for symbol in symbols_by_close_count:
    uniform_spacing = all(np.diff(df.loc[idx[symbols_by_close_count[-1],:]].index)==
        np.diff(df.loc[idx[symbols_by_close_count[-1],:]].index)[0])
    print(f'{symbol}:  {uniform_spacing}')

In [ ]:
df.loc[idx[symbols_by_close_count[1],:]].head(6)

#### How many rows of data does each symbol have?

In [ ]:
df.groupby(level=0).count().sort_values(by='close')['close'].head(10)

In [ ]:
df.info()

### Clean & Filter data

#### Infer market days and market hours from df
Data supplied contains market and after hours data. Identifying market days and market hours enables segregation of market and non-market hours data.

In [ ]:
def aggregate_daily_trade_count_from_df(df, n_symbols=5, min_trade_count_pct = 0.025):
    """Returns numerical indices for only df records containing dates with measurable market activity
    (df record count > min_trade_count_pct).  
    n_symbols: the minimum number of symbols in the df
    that are considered out of all symbols in df.
    min_trade_count_pct:  the minimum normalized count percentage of historical trades (i.e. total
    aggregated trade count for a given hour divided my max trade count of any hour) needed
    to distinguish low volume trading hours (i.e. outside market hours) from high volume
    trading hours"""
#     create idx for mutlti-index slicing
    idx = pd.IndexSlice
#     find symbols with highest rates of complete data
    symbols = df.index.get_level_values(0).unique().to_list()
    counts = [df.loc[idx[symbol,:],'close'].count() for symbol in symbols]
    lengths = [df.loc[idx[symbol,:],'close'].shape[0] for symbol in symbols]
    pct_completes = [count/length for count, length in zip(counts, lengths)]
#     obtain indices of largest pct_completes values, then analyze top n symbols 
    pct_ranked_idx = sorted(range(len(pct_completes)), key=lambda x: pct_completes[x],reverse=True)
    analysis_symbols = [symbols[x] for x in pct_ranked_idx][:min(n_symbols,len(symbols))]
#     Get aggregate count of transactions by hour of day then normalize values
    agg_trade_count_by_day = df.loc[idx[analysis_symbols,:],'close']\
        .groupby(df.loc[idx[analysis_symbols,:],'close'].index.get_level_values(1).date).count()
    return agg_trade_count_by_day

In [ ]:
agg_trade_count_by_day = aggregate_daily_trade_count_from_df(df, n_symbols=5, min_trade_count_pct = 0.025)
agg_trade_count_by_day

In [ ]:
# agg_trade_count_by_day.plot(kind='bar',figsize=(12,5))
agg_trade_count_by_day.plot(kind='hist', bins=50,figsize=(12,3), grid='both')
plt.xlabel('Daily Trade Count')
plt.show()

In [ ]:
def market_days_from_df(agg_trade_count_by_day):
    """Function assumes any dates missing in input df are days where the 
    market is closed.  Returns datetime series of market days"""
    return pd.to_datetime(agg_trade_count_by_day.index)

In [ ]:
market_dates = market_days_from_df(agg_trade_count_by_day)

In [ ]:
market_dates

### Determine Market Hours from source data

In [ ]:
def infer_market_hours_from_df(df, n_symbols=5, min_trade_count_pct = 0.2):
    """Returns 'market_open' and 'market_close' from multi-index df containing full-day 
    time-series data.  
    n_symbols: the minimum number of symbols in the df
    that are considered out of all symbols in df.
    min_trade_count_pct:  the minimum normalized count percentage of historical trades (i.e. total
    aggregated trade count for a given hour divided my max trade count of any hour) needed
    to distinguish low volume trading hours (i.e. outside market hours) from high volume
    trading hours"""
#     create idx for mutlti-index slicing
    idx = pd.IndexSlice
#     find symbols with highest rates of complete data
    symbols = df.index.get_level_values(0).unique().to_list()
    counts = [df.loc[idx[symbol,:],'close'].count() for symbol in symbols]
    lengths = [df.loc[idx[symbol,:],'close'].shape[0] for symbol in symbols]
    pct_completes = [count/length for count, length in zip(counts, lengths)]
#     obtain indices of largest pct_completes values, then analyze top n symbols 
    pct_ranked_idx = sorted(range(len(pct_completes)), key=lambda x: pct_completes[x],reverse=True)
    analysis_symbols = [symbols[x] for x in pct_ranked_idx][:min(n_symbols,len(symbols))]
#     Get aggregate count of transactions by hour of day then normalize values
    agg_trade_count_by_time = df.loc[idx[analysis_symbols,:],'close']\
        .groupby(df.loc[idx[analysis_symbols,:],'close'].index.get_level_values(1).time).count()
    max_count = agg_trade_count_by_time.max()
    normed_agg_trade_count_by_time = agg_trade_count_by_time/max_count
    # Identify first and last hour where aggregate transaction count > min_trade_count_pct
    times_w_count_gt_threshold = [idx for idx in normed_agg_trade_count_by_time.index if normed_agg_trade_count_by_time[idx] > min_trade_count_pct  ]
    market_open, market_close = times_w_count_gt_threshold[0],times_w_count_gt_threshold[-1]
    return market_open, market_close


In [ ]:
market_open, market_close = infer_market_hours_from_df(df, n_symbols=10)
market_open, market_close

#### Create a perfect, complete datetime index to reindex raw data

In [ ]:
datetime_idx_complete = pd.date_range(start=df.index.levels[1].min(), end=df.index.levels[1].max(), freq=df.index.levels[1].freq)
datetime_idx_complete

#### Are there duplicate timestamps within a given symbol's index?

In [ ]:
def check_for_duplicated_dt_indices(df):
    print('Does symbol have duplicated timestamps in datetime index?')
    symbol_dt_index_duplicated = []
    duplicated_dt_index = {}
    symbols = df.index.get_level_values(0).unique()
    for symbol in symbols:
        duplicated_dt_index = df.loc[idx[symbol,:]].index.has_duplicates
        print(f'{symbol}: {duplicated_dt_index}')    
        symbol_dt_index_duplicated.append(duplicated_dt_index)
    duplicated_dt_index = dict(zip(symbols,symbol_dt_index_duplicated ))
    return duplicated_dt_index

In [ ]:
duplicated_dt_index = check_for_duplicated_dt_indices(df)

### Fill gaps in datetime index

In [ ]:
# just in case....
df_backup = df.copy()
# df = df_backup.copy()

In [ ]:
def fill_OHLCV_gaps(df):
    """Returns complete time series from input OHLCV time series dataframe.
    Input dataframe must have timezone aware datetime index with a defined 
    frequency and can be single or dual indexed (ex. ['symbol', 'timestamp']).   
    NaN records leading up to first non-NaN record are returned unaltered.
    
    The OHLC data 
    is forward filled and NaNs in the volume data should be filled
    with '0'. This is because NaNs in price data should carry forward the
    last prices - the assumption is that if there are no new transactions 
    (i.e. volume = 0) then the most recent valid price is also the current
    valid price. Volume NaNs should be filled with '0' because if there were
    no prices or volume reported for a given timestamp it means there was no volume.
    Goal is to revise time series so that no records are NaN in the time series and
    no timestamp gaps in the time series sequence."""
    
    OHLC_cols = ['open','high','low','close']
    
    def process_df(df, symbol, datetime_idx_complete):
        """Different fill logic is needed for OHLC and volume data. 
        The OHLC data should be forward filled and NaNs in the volume
        data should be filled with '0'. This is because NaNs in price
        data should carry forward the last prices - the assumption is
        that if there are no new transactions (volume = 0) than the most 
        recent valid price is also the current valid price. Volume NaNs 
        should be filled with '0' because if there were no prices or volume
        reported for a given timestamp it means there was no volume.
        
        Leading NaN OHLC records with 0 volume remain NaN - that is OK as 
        we don't want to back fill leading NaNs.  It's better to not have 
        the data and omit the first n records than to force assumed values 
        on leading data.

        It can be seen that OHLC prices for 0-volume intervals are not NaN 
        and remain constant over intervals until volume > 0 - this is good.  """
    #             create df single index df and reindex to achieve complete datetime index
        df = df.reindex(datetime_idx_complete)
#                 fill NaN in volume columns with 0

        df['volume'] = df['volume'].fillna(0) 
#                 forward fill OHLC columns
        df[OHLC_cols] = df[OHLC_cols].ffill()
        return df
        
#     check if df is a multi-index
    is_multi_idx = isinstance(df.index, pd.MultiIndex)
    
    if is_multi_idx:
        idx = pd.IndexSlice
    #     generate a complete datetime index for dates in between min and max dates
#     this will be the new datetime index
        datetime_idx_complete = pd.date_range(start=df.index.levels[1].min(),
                                          end=df.index.levels[1].max(),
                                          freq=df.index.levels[1].freq)
        symbols = df.index.levels[0].sort_values().unique()
#         dictionary to store multiple dfs
        symbols_dict = {}
        for symbol in symbols:
            symbols_dict[symbol] = process_df(df.loc[idx[symbol,:]], symbol, datetime_idx_complete)
# combine all dfs stored in symbols_dict as multi-index
        df = pd.concat(symbols_dict)
            
    else:
        print('Function Not Ready Yet')
        datetime_idx_complete = pd.date_range(start=df.index[0],
                                          end=df.index[-1],
                                          freq=df.index.freq)
        df = process_df(df, symbol, datetime_idx_complete)
    
    return df

In [ ]:
df = fill_OHLCV_gaps(df)

In [ ]:
df.isna().sum()

In [ ]:
for symbol in df.index.levels[0].unique():
    df.loc[idx[symbol], 'close'].plot(figsize=(12,4), grid='both')
    plt.ylabel('close ($)')
    plt.title(f'{symbol} - {start_date} to {end_date}')
    print(symbol)
    plt.show()

In [ ]:
df.tail()

#### How many rows of a reindexed time series are NaN?

In [ ]:
# df.loc[idx['ACSI',:]].reindex(datetime_idx_complete).isna().sum(axis=1).value_counts()
df.loc[idx[symbols_by_close_count[0],:]].reindex(datetime_idx_complete).isna().sum(axis=1).value_counts().head(6)

In [ ]:
df.isna().sum()

### Slice df to market hours only

In [ ]:
def slice_to_market_hours(df, market_open, market_close):
    from_ = market_open.strftime("%H:%M")
    to_ = market_close.strftime("%H:%M")
    idxs = df.index.get_level_values(1).indexer_between_time(from_, to_)
    return df.iloc[idxs]

In [ ]:
display(df.shape)
df_mkt_hours = slice_to_market_hours(df, market_open, market_close)
df_mkt_hours.shape

In [ ]:
pd.Series(df_mkt_hours.index.get_level_values(1).hour).agg([min, max]).values

In [ ]:
ax = df.index.get_level_values(1).hour.value_counts().sort_index().plot(kind='bar', figsize=(12,4), grid='y', label='Timestamp Count')
x_min_mkt, x_max_mkt = pd.Series(df_mkt_hours.index.get_level_values(1).hour).agg([min, max]).values
# plt.fill_betweenx(1, )
plt.axvspan(x_min_mkt*0.98, x_max_mkt*1.02, color='green', alpha=0.2, label='Market Hours')
plt.xlabel('hour')
plt.title(f'Aggregate count of index timestamps by hour for {df.index.get_level_values(0).nunique()} symbols')
y_min, y_max = df.index.get_level_values(1).hour.value_counts().agg([min,max]).values * [0.99,1.01]
plt.ylim(y_min, y_max)
plt.legend()
plt.show()

In [ ]:
df_mkt_hours.sample(10)

In [ ]:
for symbol in df_mkt_hours.index.levels[0].unique():
    df_mkt_hours.loc[idx[symbol], 'close'].plot(figsize=(12,4))
    plt.ylabel('close')
    plt.title(f'{symbol}')
    plt.show()

### Slice df to market days only

In [ ]:
def infer_market_days(df, n_symbols=10):
    """Distinguish market days from non-market (market closed days) based on daily volumes.
    Daily mean volumes are divided in to a number of bins that is equal to the number of weeks
    in the data.  The upper edge of the lowest volume bin is used to define the threshold between 
    active market volume and a closed market.

    Function is intended for multi-index dfs with data from the same market.  Do not use
    for a mix of markets.   The more symbols with more volume in the df, the more reliable
    the result.  It is assumed that all symbols in df are in the same market/exchange and
    all follow the same 'market open' dates and times"""
    from numpy import histogram
    idx = pd.IndexSlice
    #     Slice df to a subset of n_symbols.  Sort symbols by count of non-NaN records and select
    # the top n_symbols by valid record count
    n_symbols = min(n_symbols,len(df.index.get_level_values(0).unique()))
    top_n_symbols = df.groupby(level=0)['open'].count()\
                    .sort_values(ascending=False).index.tolist()[:n_symbols]
    #     want to identify days where market is closed, (including holidays) so resample to days
    df_daily_volume = df.loc[idx[top_n_symbols,:], 'volume'].resample('D', level=1).mean()
    n_weeks = df_daily_volume.shape[0]//7
    hist_ = histogram(df_daily_volume, bins=n_weeks)
    #     This is the right hand edge of the first histogram bin
    volume_threshold = hist_[1][1]
    # identify days with volume > volume_threshold as market_open_days
    volume_GT_threshold = df_daily_volume > volume_threshold
    market_open_days = df_daily_volume.index.date[volume_GT_threshold]
    # set up filter to reduce df to only the days in market_open_days
    # which dates are market_open_dates?
    market_open_bool = pd.Series(df.index.get_level_values(1).date).isin(market_open_days)
#     OPTIONAL:  compare output df dates to market_open_days:
# pd.Series(sorted(list(set(df_mkt.index.get_level_values(1).floor('D')\
# .date))) == market_open_days).value_counts()
    return df.loc[idx[market_open_bool.values,:]]

In [ ]:
%%time
display(df_mkt_hours.shape)
df_mkt = infer_market_days(df_mkt_hours)
df_mkt.index.names = ['symbol', 'timestamp']
df_mkt.shape

In [ ]:
df_mkt.sample(10)

In [ ]:
df_mkt.loc[df_mkt['volume']>0, 'volume'].hist(bins=73)
plt.yscale('log')
plt.xlabel('Trade Volume')
plt.title(f'Distribution of Volume by tick for non-zero '
          f'volume ticks for all {df_mkt.index.levels[0].nunique()} symbols')
plt.show()

In [ ]:
for symbol in df_mkt.index.levels[0].unique():
    df_mkt.loc[idx[symbol], 'close'].plot(figsize=(12,4))
    plt.ylabel('close')
    plt.title(f'{symbol}')
    plt.show()

### How Many NaNs remain?
At this point it is expected that only blocks of leading NaNs could be present.  Drop these and effectively remove the beginning of time series history for specific stocks. 

In [ ]:
df_mkt.isna().sum()

In [ ]:
df_mkt.loc[df_mkt.isna().any(1)][:8]

In [ ]:
df_mkt.isna().sum(1).plot()

In [ ]:
df_mkt.dropna(inplace=True, subset=OHLC_cols)
df_mkt.loc[df_mkt.isna().any(1)][:8]

In [ ]:
df_mkt.isna().sum()

### Filter Noise in Data
Remember that noise filtering steps will also need to be applied to incoming data in live strategies.  For this reason, the approach here is to correct only blatant extreme values.  Otherwise noise will be left in the data.

In [ ]:
for symbol in df_mkt.index.levels[0].unique():
#     df[OHLC_cols].replace(0, np.nan).bfill().where(df_mkt.ffill().notnull())
    df_mkt.loc[idx[symbol], 'close'].plot(figsize=(12,4), color='blue', label='raw zeros')
    df_mkt.loc[idx[symbol], 'close'].replace(0, np.nan).bfill().where(df_mkt.ffill().notnull()).plot(figsize=(12,4), color='red', label='fill zeros')
    plt.title(f'{symbol}')
    plt.xlabel('close')
    plt.legend()
    plt.show()

#### Compare methods of filling 0's - Interpolate seems best

In [ ]:
CAT_close_2021_03 = df_mkt.loc[idx['CAT',"2021-03-02":"2021-03-04"], 'close']
CAT_close_2021_03.plot(label='raw zeros')
CAT_close_2021_03.replace(0, np.nan).bfill().where(CAT_close_2021_03.ffill()\
           .notnull()).plot(figsize=(12,4), color='red', label='filled zeros').plot()
CAT_close_2021_03.replace(0,np.nan).interpolate(limit_area='inside')\
            .plot(label='interpolated', color='green', marker='x')
# CAT_close_2021_03.replace(0, np.nan).groupby('timestamp').transform(pd.DataFrame.interpolate)
# plt.xticks(ticks=np.arange(0,CAT_close_2021_03.shape[0]), labels = CAT_close_2021_03.index.levels[1], rotation = 75)
plt.xticks(rotation = 75)
plt.ylim(CAT_close_2021_03.quantile(0.5)*0.95, CAT_close_2021_03.quantile(0.5)*1.05)
plt.legend()
plt.grid('both')
plt.show()

### Fill zeros in OHLC of df_mkt and save as df_denoised

In [ ]:
(df_mkt[OHLC_cols] == 0).sum()

In [ ]:
dfs_no_zero = {}
symbols = df_mkt.index.levels[0].unique()
# slice simple index df's out of multi-index df
for symbol in symbols:
    dfs_no_zero[symbol] = df_mkt.loc[idx[symbol], OHLC_cols].replace(0,np.nan).interpolate(limit_area='inside')
#     dfs_no_zero[symbol] = df_mkt.loc[idx[symbol], OHLC_cols].replacbe(0,np.nan).interpolate(method='index', limit_area='inside')
#     print(symbol)
df_denoised = pd.concat(dfs_no_zero,keys=symbols).assign(volume = df_mkt['volume'])
del dfs_no_zero
df_denoised['volume'] = df_mkt['volume'].copy()
# df_denoised.describe()

In [ ]:
(df_denoised[OHLC_cols] == 0).sum()

#### Compare Raw OHLC data to denoised data

In [ ]:
for symbol in df_mkt.index.levels[0].unique():
    df_mkt.loc[idx[symbol], 'close'].plot(figsize=(12,4), color='blue', label='raw')
#     df_mkt.loc[idx[symbol], 'close'].replace(0, np.nan).groupby('timestamp')\
#         .transform(pd.DataFrame.interpolate, limit_area='inside')\
#         .plot(figsize=(12,4), color='orange', linestyle = '--')
    df_denoised.loc[idx[symbol], 'close'].plot(figsize=(12,4), color='black', lw=1.5, linestyle='--', label='denoised')
    plt.legend()
    print(symbol)
    plt.show()

In [ ]:
df_mkt.loc[idx['TMAT'], 'close'].replace(0, np.nan).groupby('timestamp')\
        .transform(pd.DataFrame.interpolate, limit_area='inside')

In [ ]:
CAT_close_2021_03 = df_mkt.loc[idx['CAT',"2021-03-02":"2021-03-04"], 'close']
CAT_close_2021_03_dn = df_denoised.loc[idx['CAT',"2021-03-02":"2021-03-04"], 'close']
CAT_close_2021_03.plot(label='raw zeros')
# CAT_close_2021_03.replace(0, np.nan).bfill().where(CAT_close_2021_03.ffill()\
#            .notnull()).plot(figsize=(12,4), color='red', label='filled zeros').plot()
CAT_close_2021_03_dn\
            .plot(label='denoised', color='green', marker='x')
# CAT_close_2021_03.replace(0, np.nan).groupby('timestamp').transform(pd.DataFrame.interpolate)
plt.xticks(rotation = 75)
plt.ylim(CAT_close_2021_03.quantile(0.5)*0.95, CAT_close_2021_03.quantile(0.5)*1.05)
plt.legend()
plt.grid('both')
plt.show()

### Check volume data

In [ ]:
df_daily = df.resample('D', level=1).mean()
df_daily

In [ ]:
df_daily['vol_div_by_99_pct'] = df_daily['volume']/df_daily['volume'].quantile(0.99)
# df_daily.groupby(df_daily.index.weekday)['vol_div_by_99_pct'].mean()
df_daily['vol_div_by_99_pct']

In [ ]:
df_daily['vol_div_by_99_pct'].quantile([0.05,0.1,0.15,0.25, 0.8])

In [ ]:
df_daily['vol_div_by_99_pct'][40:110].plot(kind='bar', figsize=(14,4))
plt.title('Daily Volume divided by 99th percentile volume')
plt.xticks(np.arange(70),df_daily['vol_div_by_99_pct'][:70].index.strftime("%m-%d-%y"))
plt.grid(axis='y')
plt.show()

In [ ]:
ax = df_daily.assign(weekday = df_daily.index.day_name()).plot(kind='scatter', x='weekday',y='volume')
ax.set_xticks(list(calendar.day_name))
plt.title('Volume by day of week')
plt.grid(axis='y')
plt.show()

In [ ]:
(df_daily['volume']/df_daily['volume'].quantile(0.99)).plot(kind='hist', bins=100)
# plt.xlim(0.1,3)
plt.title('Distribution of Daily Volume divided by 99th percentile volume')
plt.show()

In [ ]:
df_daily['Percent_Change'] = df_daily[['open','close']].pct_change(axis=1).iloc[:,-1]
df_daily.head(3)

In [ ]:
df_daily[['Percent_Change','vol_div_by_99_pct']].corr()

In [ ]:
df_daily[['Percent_Change','volume']].plot(kind='scatter',
                          x='Percent_Change', y='volume')
plt.xlim(-0.01, 0.01)
plt.title('Daily Open to Close Percent Change vs. Volume')
plt.show()

In [ ]:
df_daily[['Percent_Change','vol_div_by_99_pct']].plot(kind='scatter',
                          x='Percent_Change', y='vol_div_by_99_pct')
plt.xlim(-0.01, 0.01)
plt.title('Daily Open to Close Percent Change vs. Relative Volume')
plt.show()

In [ ]:
df_denoised

## Data Cleaning Complete

## Add Features (Technical Indicators)
https://github.com/twopirllc/pandas-ta

https://github.com/bukosabino/ta

In [ ]:
df.index.levels[0].tolist()

In [ ]:
symbol = 'CAT'
# df = df_denoised.copy()
df = df_denoised.loc[symbol].dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

### Calculate Returns over various periods
To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method `.pct_change(n_periods)`, that is, returns over various periods as identified by lags.

We then convert the wide result back to long format with the `.stack()` method, use `.pipe()` to apply the `.clip()` method to the resulting `DataFrame`, and winsorize returns at the [1%, 99%] levels; that is, we cap outliers at these percentiles.

Finally, we normalize returns using the geometric average. After using `.swaplevel()` to change the order of the `MultiIndex` levels, we obtain compounded periodic returns for six periods ranging from 1 to 12 months:

In [ ]:
def log_return(df, n_periods, start, end):
    """Example:  
    df['log_return'] = log_return(df, 1, 'open','close')"""
    return np.log(df[end]/df[start].shift(n_periods))

In [ ]:
df['log_return'] = log_return(df, 0, 'open','close')

In [ ]:
fig, [ax1, ax2] = plt.subplots(2,1, sharex=True)

df['close'].plot(ax=ax1, color = 'black')
ax1.set_title('close')
df['log_return'].cumsum().plot(ax=ax2, color='orange')
ax2.set_title('Log Return')
plt.show()

# stopped Here 1-15-21

In [ ]:
outlier_cutoff = 0.01
data = df.copy()
lags = [1, 2, 3, 6, 9, 12]
for lag in lags:
    data[f'log_return_{lag}'] = (data
                           .pct_change(lag)
                           .stack()
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
data = data.swaplevel().dropna()
data.info()

In [ ]:
?? ta.log_return

In [ ]:
df_denoised.ta.log_return()

In [ ]:
# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

In [ ]:
# Clean NaN values
# df = dropna(df)

# Add all ta features
df_features = add_all_ta_features(df_denoised,open="open",
          high="high", low="low", close="close", volume="volume")

In [ ]:
df_features.sample(5).T

In [ ]:
ta

## Apply Machine Learning

In [ ]:
df_denoised.info()

In [ ]:
df_features.info()

### Principal Components Analysis

# NOTES

#### De-Noise OHLC data

In [ ]:
from pykalman import KalmanFilter

In [ ]:
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

In [ ]:
state_means, _ = kf.filter(sp500)

##### Compare with moving average

In [ ]:
sp500_smoothed = sp500.to_frame('close')
sp500_smoothed['Kalman Filter'] = state_means
for months in [1,2,3]:
    sp500_smoothed[f'MA ({months}m)'] = sp500.rolling(window=months*21).mean()

ax = sp500_smoothed.plot(title='Kalman Filter vs Moving Average', figsize=(14,6), lw=1, rot=0)
ax.set_xlabel('')
ax.set_ylabel('S&P 500')
plt.tight_layout()
sns.despine();

In [ ]:
def remove_incomplete_symbols(min_threshold_pct=20):
    """Return multi-index df where all symbols whose history has data for less
    than the min_threshold_pct is omitted"""

### Visualize data

In [ ]:
import matplotlib as mpl

In [ ]:
# df.loc[idx[assets_query[0],:]].groupby(pd.Grouper(freq='1Min'))
df.loc[idx[assets_query[0],:]].groupby(pd.Grouper(freq='1D')).agg({'open' : 'mean', 'high': 'mean','low': 'mean','close': 'mean','volume':'sum'})
# .groupby(pd.Grouper(freq=freq,level=1)).agg({'open' : 'mean', 'high': 'mean','low': 'mean','close': 'mean','volume':'sum'})

In [ ]:
def price_volume(df, price='open', vol='volume', symbol=assets_query[0], freq='1Hour',fname=None):
    # for multiindex:
    idx = pd.IndexSlice
    df = df.loc[idx[symbol,:]].groupby(pd.Grouper(freq=freq)).agg({'open' : 'mean',
                                                'high': 'mean',
                                                'low': 'mean',
                                                'close': 'mean',
                                                'volume':'sum'})
    
#     df = df.groupby(pd.Grouper(freq=freq)).agg({'open' : 'mean',
#                                                 'high': 'mean',
#                                                 'low': 'mean',
#                                                 'close': 'mean',
#                                                 'volume':'sum'})
#     df.fillna(method='ffill', inplace=True)
    
    fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(15,8))
    axes[0].plot(df.index, df[price])
    axes[1].bar(df.index, df[vol], width=1/(5*len(df.index)), color='r')
    
    # formatting
    xfmt = mpl.dates.DateFormatter('%H:%M')
    axes[1].xaxis.set_major_locator(mpl.dates.HourLocator(interval=3))
#     axes[1].xaxis.set_major_formatter(xfmt)
#     axes[1].get_xaxis().set_tick_params(which='major', pad=25)
    axes[0].set_title(f'{price}', fontsize=14)
    axes[1].set_title('Volume', fontsize=14)
#     fig.autofmt_xdate()
    fig.suptitle(symbol)
#     fig.tight_layout()
#     plt.subplots_adjust(top=0.9);

In [ ]:
price_volume(df, freq='1D')

In [ ]:
df.loc[idx[assets_query[0],:]].groupby(pd.Grouper(freq='1D')).agg({'open' : 'mean', 'high': 'mean','low': 'mean','close': 'mean','volume':'sum'}).describe()

In [ ]:
def get_bar_stats(agg_trades, weighted_price='close'):
#     vwap = agg_trades.apply(lambda x: np.average(x[weighted_price], weights=x.volume)).to_frame('vwap')
#     ohlc = agg_trades.price.ohlc()
    vol = agg_trades.shares.sum().to_frame('vol')
#     txn = agg_trades.shares.size().to_frame('txn')
    return pd.concat([ohlc, vwap, vol, txn], axis=1)

In [ ]:
df.loc[idx[assets_query[0],:],].droplevel(0)

In [ ]:
import numpy as np
trades = df.loc[idx[assets_query[0],:],].droplevel(0)
resampled = trades.groupby(pd.Grouper(freq='1H'))
time_bars = get_bar_stats(resampled)
# normaltest(time_bars.vwap.pct_change().dropna())

In [ ]:
price_volume(time_bars, 
             suptitle=f'Time Bars | {stock} | {pd.to_datetime(date).date()}', 
             fname='time_bars')

In [ ]:
import bokeh
bokeh.sampledata.download()

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.sampledata.stocks import MSFT
output_notebook()
MSFT = pd.DataFrame(MSFT)[:50]
#     df["date"] = pd.to_datetime(df["date"])
MSFT.head()

In [ ]:
pd.to_datetime(MSFT['date'])

In [ ]:
df.loc[idx[assets_query[0],:]].head(2)

In [ ]:
df.loc[idx[assets_query[0],:]].index.strftime("%Y-%m-%d")

In [ ]:
df.loc[idx[assets_query[0],:]].isna()

In [ ]:

def candlestick_w_volume(df, symbol):
    """Plot OHLC Candlestick with Volume chart"""
    from math import pi
#     import pandas as pd
    from bokeh.plotting import figure, output_notebook, show
    from bokeh.layouts import gridplot
    from bokeh.sampledata.stocks import MSFT
    output_notebook()
#     df = pd.DataFrame(MSFT)[:50]
#     df["date"] = pd.to_datetime(df["date"]) - Original Code
# for multiindex:
    df = df.loc[idx[symbol,:]]
    
    df["date"] = df.index.strftime("%Y-%m-%d")
    display(df.head(2))
    

    inc = df.close > df.open
    dec = df.open > df.close
    w = 12*60*60*1000 # half day in ms
    print(inc, dec)

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p1 = figure(x_axis_type="datetime", tools=TOOLS, plot_width=700, plot_height=300, title = f"{symbol}")
    p1.xaxis.visible = False
    p1.xaxis.major_label_orientation = pi/4
    p1.grid.grid_line_alpha=0.3
    print(p1.x_range)
    
    p1.segment(df.date, df.high, df.date, df.low, color="black")
    p1.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p1.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p2 = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=700, plot_height=200, x_range=p1.x_range)
    p2.xaxis.major_label_orientation = pi/4
    p2.grid.grid_line_alpha=0.3
    p2.vbar(df.date, w, df.volume, [0]*df.shape[0])

    show(gridplot([[p1],[p2]]))

In [ ]:
candlestick_w_volume(df, assets_query[0])

In [ ]:
def candlestick_w_volume():
    """Plot OHLC Candlestick with Volume chart"""
    from math import pi

    import pandas as pd

    from bokeh.plotting import figure, output_notebook, show
    from bokeh.layouts import gridplot
    from bokeh.sampledata.stocks import MSFT

    output_notebook()

    df = pd.DataFrame(MSFT)[:50]
    df["date"] = pd.to_datetime(df["date"])

    inc = df.close > df.open
    dec = df.open > df.close
    w = 12*60*60*1000 # half day in ms

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p1 = figure(x_axis_type="datetime", tools=TOOLS, plot_width=700, plot_height=300, title = "MSFT Candlestick with Volume")
    p1.xaxis.visible = False
    p1.xaxis.major_label_orientation = pi/4
    p1.grid.grid_line_alpha=0.3

    p1.segment(df.date, df.high, df.date, df.low, color="black")
    p1.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p1.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p2 = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=700, plot_height=200, x_range=p1.x_range)
    p2.xaxis.major_label_orientation = pi/4
    p2.grid.grid_line_alpha=0.3
    p2.vbar(df.date, w, df.volume, [0]*df.shape[0])

    show(gridplot([[p1],[p2]]))

In [ ]:
candlestick_w_volume()

### Save df to disk

In [ ]:
def multi_symbol_df_to_csv(df, directory=''):
    start_date = datetime.strftime(df.index.get_level_values(1).min(),"%Y-%m-%d")
    end_date = datetime.strftime(df.index.get_level_values(1).max(),"%Y-%m-%d")
    n_symbols = df.index.get_level_values(0).nunique()
    frequency = df.index.levels[1].freqstr
    filename = os.path.join(directory,f'symbols_{n_symbols}_{frequency}_{start_date} to {end_date}.csv') 
    df.to_csv(filename)
    print(f'df saved as "{filename}"')

In [ ]:
multi_symbol_df_to_csv(df, directory='data')

## Add Technical indicators
normalized volume?

In [ ]:
import alpaca_backtrader_api
import backtrader as bt
# from datetime import datetime

# ALPACA_API_KEY = <key_id>
# ALPACA_SECRET_KEY = <secret_key>
# ALPACA_PAPER = True


class SmaCross(bt.SignalStrategy):
  def __init__(self):
    sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
    crossover = bt.ind.CrossOver(sma1, sma2)
    self.signal_add(bt.SIGNAL_LONG, crossover)


cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

In [ ]:
store = alpaca_backtrader_api.AlpacaStore(
#     key_id=ALPACA_API_KEY,
#     secret_key=ALPACA_SECRET_KEY,
#     paper=ALPACA_PAPER
)

In [ ]:
if not ALPACA_PAPER:
  broker = store.getbroker()  # or just alpaca_backtrader_api.AlpacaBroker()
  cerebro.setbroker(broker)

In [ ]:
DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
data0 = DataFactory(dataname='AAPL', historical=True, fromdate=datetime(
    2015, 1, 1), timeframe=bt.TimeFrame.Days)
cerebro.adddata(data0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot()

### Technical Indicator functions

# END